In [39]:
import torch
import os
from transformers import AutoModel, AutoTokenizer
from TweetNormalizer import normalizeTweet
from huggingface_hub import hf_hub_download
import os
import csv
import sys
import re
import math
import regex
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import xml.dom.minidom
from xml.dom.minidom import parse
from xml.etree import ElementTree
import xmltodict
from lxml import etree
from bs4 import BeautifulSoup
import transformers
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import emoji
import demoji
#from emoji.unicode_codes import UNICODE_EMOJI
import emot as e
#from emot.emo_unicode import UNICODE_EMOJI, UNICODE_EMOJI_ALIAS, EMOTICONS_EMO
#from emoji_translate.emoji_translate import Translator
from html import unescape
import html
import unicodedata
from googletrans import Translator
import numpy as np
import pandas as pd
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
import xml.dom.minidom
from xml.dom.minidom import parse
from xml.etree import ElementTree
import xmltodict
from lxml import etree
from bs4 import BeautifulSoup
import transformers
from transformers import AutoModel, BertTokenizerFast
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import zipfile as zf
from html import unescape
import html
import unicodedata
import xml.etree.ElementTree
import bs4
import os
import xml.etree.ElementTree as ET
import csv
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from kneed import KneeLocator
import pickle
import patoolib
import torch
from transformers import AutoTokenizer
from palm_rlhf_pytorch import PaLM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
import warnings
from collections import Counter
from sklearn.cluster import OPTICS, cluster_optics_dbscan
from artlearn import FuzzyART
import glob
import gc
import tensorflow as tf
from sklearn.metrics import precision_score ,recall_score , f1_score
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical 
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

# DataSet

In [6]:
patoolib.extract_archive("pan22-author-profiling-training-2022-03-29-1.zip")

patool: Extracting pan22-author-profiling-training-2022-03-29-1.zip ...
patool: ... pan22-author-profiling-training-2022-03-29-1.zip extracted to `pan22-author-profiling-training-2022-03-29-14' (local file exists).


'pan22-author-profiling-training-2022-03-29-14'

In [7]:
files = os.listdir('./pan22-author-profiling-training-2022-03-29/en')
files.remove('truth.txt')

In [8]:
data = list()
for file in files:
    if not file.startswith('.'):
        ID = 0
        tree = ET.parse(f'./pan22-author-profiling-training-2022-03-29-1/pan22-author-profiling-training-2022-03-29/en/{file}')
        root = tree.getroot()
        docs = root.find('documents')
        for doc in docs.findall('document'):
            ID += 1
            row = [file, ID, doc.text] # add bert
            #print(row)
            data.append(row)

In [9]:
fields = ['author_name', 'tweet_id', 'tweet'] 
with open('Clusterdataset.csv', 'w') as f:
    write = csv.writer(f)      
    write.writerow(fields)
    write.writerows(data)

In [10]:
df = pd.read_csv('./Clusterdataset.csv')

# preporcessing

In [11]:
with open("twitterSlang/twitterSlang.csv", "r") as f:
    reader = csv.reader(f,delimiter=',')
    slangTwitter = {row[0]:row[1] for row in reader}

In [12]:
def replaceTwitterSlang(tweet):
    """
    Replaces twitter specific slang, based on a dictionary created from various online sources
    """
    tokens = tweet.split(' ')
    return ' '.join([slangTwitter[token.lower()] if token.lower() in slangTwitter else token for token in tokens])


In [13]:
def remove_control_char(txt):

    txt = str(txt).strip()
    txt = "".join(ch for ch in txt if unicodedata.category(ch)!="Cc")
    txt = re.sub(r'[\u200d\u200f\u202b\x01]', '', txt)
    txt = re.sub(r'(\u200c)+', r'\1', txt)
    txt = re.sub(r'(\s)+\u200c', r'\1', txt)
    txt = re.sub(r'\u200c(\s)+', ' ', txt)
    txt = re.sub(r'([0-9,\.\(\)\[\];،؛,])\u200c', r'\1', txt)
    txt = re.sub(r'\u200c([0-9,\.\(\)\]\[;،؛,])', r'\1', txt)
    txt = re.sub(r'^[\s\u200c]*(.*?)[\s\u200c]*$', r'\1', txt)
    return txt

In [14]:
stop = set(stopwords.words('english'))

In [15]:
def rem_en(input_txt):
    words = input_txt.lower().split()
    noise_free_words = [word for word in words if word not in stop] 
    noise_free_text = " ".join(noise_free_words) 
    return noise_free_text

In [16]:
def seprate_twits(txt):
    txt = str(txt).replace("|||",".\n") #todo : why?
    txt = txt.replace("|","")
    txt = re.sub(r' +', ' ', txt)
    txt = re.sub(r' +\.', '.', txt)
    return txt

In [17]:
#deep clean
df["CLEAN_TEXT"] = df["tweet"].apply(replaceTwitterSlang)
df["CLEAN_TEXT"] = df["CLEAN_TEXT"].apply(lambda s: str(s).lower())
df["CLEAN_TEXT"] = df["CLEAN_TEXT"].apply(lambda s: html.unescape(str(s)))
df["CLEAN_TEXT"] = df["CLEAN_TEXT"].apply(lambda s: remove_control_char(s))
df["CLEAN_TEXT"] = df["CLEAN_TEXT"].apply(lambda s: re.sub("(\w+://\S+)", " ", s))
df["CLEAN_TEXT"] = df["CLEAN_TEXT"].replace('#user#', '', regex=True)
df["CLEAN_TEXT"] = df["CLEAN_TEXT"].replace('#hashtag#', '', regex=True)
df["CLEAN_TEXT"] = df["CLEAN_TEXT"].replace('#url#', '', regex=True)
df["CLEAN_TEXT"] = df["CLEAN_TEXT"].apply(lambda s: rem_en(s))
df["CLEAN_TEXT"] = df["CLEAN_TEXT"].apply(lambda s: ' '.join(re.sub("[\*•.,!?:;\-\+='...\"@#_%&\$…]", " ", s).split()))
df["CLEAN_TEXT"] = df["CLEAN_TEXT"].apply(lambda s: ' '.join(re.sub("[“”’‘]", "", s).split()))
df["CLEAN_TEXT"] = df["CLEAN_TEXT"].replace('(number\s+)+', 'number ', regex=True)
df["CLEAN_TEXT"] = df["CLEAN_TEXT"].replace(r' +', ' ', regex=True)
df["CLEAN_TEXT"] = df["CLEAN_TEXT"].apply(lambda s: seprate_twits(s))
df["CLEAN_TEXT"] = df["CLEAN_TEXT"].replace(r'[^a-z\s\.]', '', regex=True)

In [19]:
for i in range(0,419):
    inputs = df.to_dict('records')[i]
    df["CLEAN_TEXT"][i] = emoji.demojize(inputs["CLEAN_TEXT"],delimiters=(" ", " "))

/tmp/ipykernel_26969/2659571774.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["CLEAN_TEXT"][i] = emoji.demojize(inputs["CLEAN_TEXT"],delimiters=(" ", " "))


In [18]:
def remove_emoji(string):
    return emoji.get_emoji_regexp().sub(u'', string)

In [20]:
# lite cleen
df["LITE_CLEAN_TEXT"] = df["tweet"].apply(replaceTwitterSlang)
df["LITE_CLEAN_TEXT"] = df["LITE_CLEAN_TEXT"].apply(lambda s: str(s).lower())
df["LITE_CLEAN_TEXT"] = df["LITE_CLEAN_TEXT"].apply(lambda s: html.unescape(str(s)))
df["LITE_CLEAN_TEXT"] = df["LITE_CLEAN_TEXT"].apply(lambda s: remove_control_char(s))
df["LITE_CLEAN_TEXT"] = df["LITE_CLEAN_TEXT"].apply(lambda s: re.sub("(\w+://\S+)", " ", s))
df["LITE_CLEAN_TEXT"] = df["LITE_CLEAN_TEXT"].replace('#user#', '', regex=True)
df["LITE_CLEAN_TEXT"] = df["LITE_CLEAN_TEXT"].replace('#hashtag#', '', regex=True)
df["LITE_CLEAN_TEXT"] = df["LITE_CLEAN_TEXT"].replace('#url#', '', regex=True)
df["LITE_CLEAN_TEXT"] = df["LITE_CLEAN_TEXT"].apply(lambda s: ' '.join(re.sub("[\*•:;\-\+=\"@#_%&\$…]", " ", s).split()))
df["LITE_CLEAN_TEXT"] = df["LITE_CLEAN_TEXT"].apply(lambda s: ' '.join(re.sub("[“”]", "", s).split()))
df["LITE_CLEAN_TEXT"] = df["LITE_CLEAN_TEXT"].apply(lambda s: re.sub("[’‘]", "'", s))
df["LITE_CLEAN_TEXT"] = df["LITE_CLEAN_TEXT"].replace(r' +', ' ', regex=True)
df["LITE_CLEAN_TEXT"] = df["LITE_CLEAN_TEXT"].apply(lambda s: seprate_twits(s))
df["LITE_CLEAN_TEXT"] = df["LITE_CLEAN_TEXT"].apply(lambda s: remove_emoji(s)) #todo: remove only emoji

/tmp/ipykernel_26969/3148388468.py:2: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  return emoji.get_emoji_regexp().sub(u'', string)


# Labels

In [21]:
res = dict()
with open('./truth.txt', 'r') as f:
    lines = f.readlines()
    for i in lines:
        name_label = i.split(':::')
        # lab = name_label[1].split('\')
        res[name_label[0]] = name_label[1][:-1]

In [22]:
labels = list()
for i in list(df['author_name']):
    lab = res[i[:-4]]
    if lab == 'I':       
        labels.append(1)
    else:
        labels.append(0)

In [23]:
df['label'] = labels

In [ ]:
df.to_csv('Cluster_clean_data.csv')

In [ ]:
df = pd.read_csv('Cluster_clean_data.csv')

In [24]:
for i in df.iterrows():
     if type(i[1][4]) == float:
            df =df.drop(i[0])

# Feature Extraction with Bertweet

In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [42]:
bertweet = AutoModel.from_pretrained("vinai/bertweet-base",return_dict=False)
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [27]:
def Dim_reduction(sentences, tokenizer, model):
    '''
        This method will accept array of sentences, roberta tokenizer & model
        next it will call methods for dimention reduction
    '''
    vecs = []
    with torch.no_grad():

        for sentence in sentences:

                
            inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True,  max_length=64)
            inputs['input_ids'] = inputs['input_ids']
            inputs['attention_mask'] = inputs['attention_mask']

            hidden_states = model(**inputs, return_dict=True, output_hidden_states=True).hidden_states

            #Averaging the first & last hidden states
            output_hidden_state = (hidden_states[-1] + hidden_states[1]).mean(dim=1)

            vec = output_hidden_state.cpu().numpy()[0]

            vecs.append(vec)


    #Finding Kernal
    kernel, bias = compute_kernel_bias([vecs])
    kernel = kernel[:, :128]
    embeddings = []
    embeddings = np.vstack(vecs)
    embeddings = transform_and_normalize(embeddings, 
                kernel=kernel,
                bias=bias
            )

    return embeddings

In [28]:
def transform_and_normalize(vecs, kernel, bias):
    """
        Applying transformation then standardize
    """
    if not (kernel is None or bias is None):
        vecs = (vecs + bias).dot(kernel)
    return normalize(vecs)

In [29]:
def normalize(vecs):
    """
        Standardization
    """
    return vecs / (vecs**2).sum(axis=1, keepdims=True)**0.5

In [30]:
def compute_kernel_bias(vecs):
    """
    Calculate Kernal & Bias for the final transformation - y = (x + bias).dot(kernel)
    """
    vecs = np.concatenate(vecs, axis=0)
    mu = vecs.mean(axis=0, keepdims=True)
    cov = np.cov(vecs.T)
    u, s, vh = np.linalg.svd(cov)
    W = np.dot(u, np.diag(s**0.5))
    W = np.linalg.inv(W.T)
    return W, -mu

In [31]:
def Dimention_Reduction(new_df):
    embeddings = Dim_reduction(list(new_df['CLEAN_TEXT']), tokenizer, bertweet)
    newembedding = list()
    for i in range(len(embeddings)):
        newembedding.append(embeddings[i])
    new_df['DM_feature'] = newembedding
    return newembedding , new_df

# Clustering

In [64]:
def clustering_each_author(newembedding , new_df):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(newembedding)
    nbrs = NearestNeighbors(n_neighbors = 5).fit(X_scaled) 
    neigh_dist, neigh_ind = nbrs.kneighbors(X_scaled)
    sort_neigh_dist = np.sort(neigh_dist, axis = 0) 
    k_dist = sort_neigh_dist[:, 4]
    kneedle = KneeLocator(x = range(1, len(neigh_dist)+1), y = k_dist, S = 1.0, 
                          curve = "concave", direction = "increasing", online=True)
    clusters = DBSCAN(eps = float(str(kneedle.knee_y)[:3]), min_samples = 3 ).fit(newembedding)
    new_df['cluster_label'] = clusters.labels_
    noise_list = list()
    for i in range(len(clusters.labels_)):
        if clusters.labels_[i] == -1 :
            noise_list.append(1)
        else:
            noise_list.append(0)
    new_df['noise'] = noise_list
    new__df = knn_removing_noise(new_df)
    return  new__df
    

# Removing noise

In [34]:
def knn_removing_noise(new_df):
    selected_columns = new_df[["DM_feature","cluster_label"]]
    without_noise_df = selected_columns.copy()
    test_set = without_noise_df[without_noise_df['cluster_label'] == -1]
    if (len(test_set) > 0 ):
        train_set = without_noise_df[without_noise_df['cluster_label'] != -1]
        x_train = train_set['DM_feature']
        y_train = train_set['cluster_label']
        x_test = test_set['DM_feature']
        y_test = test_set['cluster_label']
        train_features = x_train.apply(pd.Series)
        train_processed = pd.concat([x_train, train_features], axis=1)
        train_processed = train_processed.drop('DM_feature', axis=1)
        knn_classifier = KNeighborsClassifier(n_neighbors = 3 )
        knn_classifier.fit(train_processed, y_train)
        test_features = x_test.apply(pd.Series)
        test_processed = pd.concat([x_test, test_features], axis=1)
        test_processed = test_processed.drop('DM_feature', axis=1)
        predicted_labels = knn_classifier.predict(test_processed)
        predicted_labels = pd.DataFrame({'predicted_label':predicted_labels})
        j = 0
        for i in range(len(new_df)):
            if (new_df['cluster_label'].iloc[i] == -1):
                new_df['cluster_label'].iloc[i] = predicted_labels.iloc[j] 
                j = j + 1 
        return new_df #todo: add one clumn to sepreate knn from clustering
    else:
        return new_df

# Feature Extraction with PaLm

In [45]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")

In [46]:
model = PaLM(
    num_tokens=50304, dim=1024, depth=24, dim_head=128, heads=8, flash_attn=False, qk_rmsnorm = False,
).to(device).eval()
checkpoint = torch.load('./PaLM/checkpoints/palm_410m_8k_v0.pt', map_location=device)
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [47]:
author_name_list = df['author_name'].unique()

In [52]:
warnings.filterwarnings('ignore')
for i in list(df['author_name'].unique()):
    try:
        new_df = df[df['author_name'] == i ]
        newembedding,new__df = Dimention_Reduction(new_df)
        df_author = clustering_each_author(newembedding, new__df)
      #  with open(f'./author_clusterrr/{i[:-4]}.pk', 'wb') as handle:
      #      pickle.dump(text_label_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(f'./Final_dimention_128/{i[:-4]}.pk', 'wb') as handle:
            pickle.dump(df_author, handle, protocol=pickle.HIGHEST_PROTOCOL)
    except:
        print(i)
        pass


In [ ]:
for num, i in enumerate(author_name_list[:]):
    with open(f'./Final_dimention_128/{i[:-4]}.pk', 'rb') as handle:
        mini_df = pickle.load(handle)
        palm_features = list()
        res = Counter(list(mini_df['cluster_label']))
    for j in range (len (mini_df)):
        encoded_input = tokenizer (mini_df['LITE_CLEAN_TEXT'].iloc[j] , return_tensors='pt')
        features = model(x = encoded_input['input_ids'], return_only_embedding=True)
        features = features.detach().numpy()
        palm_features.append(features)
    mini_df['palm_features'] = palm_features
    del palm_features
    with open(f'./author_dffff_palm/{i[:-4]}.pk', 'wb') as handle:
        pickle.dump(mini_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [53]:
all_data = list()
pickle_files = glob.glob ('author_dffff_palm/*.pk')
merged_df = pd.DataFrame()
for pickle_file in pickle_files[:]:
    final_df = pd.read_pickle(pickle_file)
    all_data.append(final_df)

In [54]:
final_df = pd.concat(all_data, ignore_index = True)

In [55]:
means_list = list()
for feature in (final_df['palm_features']):
    f = torch.from_numpy(feature)
  #  print(type(feature))
    means_list.append(torch.mean(f , 1))

In [56]:
final_df['mean_palm'] = means_list

In [58]:
cluster_size = final_df.groupby (['author_name','cluster_label']).size().reset_index(name = 'cluster_size')
final_df= pd.merge(final_df,cluster_size , on = ['author_name','cluster_label'], how = 'left')
final_df['mean_palm'] = final_df['mean_palm'].apply(lambda x: x.numpy())
feature_aggregates = final_df.groupby(['author_name','cluster_label']).agg( feature_avg = ('mean_palm', 'mean')).reset_index()
final_df = pd.merge(final_df, feature_aggregates , on = ['author_name','cluster_label'], how = 'left')
final_df = final_df.drop(columns = ['tweet_id'])
final_df = final_df.drop_duplicates(subset=['author_name','cluster_label'], keep='first')

In [59]:
authorlist = df['author_name'].unique()

In [60]:
avg_list = list()
for i in authorlist:
    a_df = final_df[final_df.author_name == i]
    sum_size = 0
    summation = 0
    for j in list(a_df.iterrows()):
        summation += j[1]['feature_avg'] * j[1]['cluster_size']
        sum_size += j[1]['cluster_size']
    avg = summation / sum_size
    avg_list.append([i, avg, j[1]['label']])
avg_df = pd.DataFrame(avg_list , columns = ['author' , 'feature' , 'label'])


In [61]:
np_feature = list()
for i in list(avg_df['feature']):
    np_feature.append(i.reshape(-1))
avg_df['np_feature'] =  np_feature

In [62]:
#splite train and test data
X = avg_df['np_feature'] 
X = np.array([np.asarray(i).astype('float32') for i in X])
y = avg_df['label']
#y = to_categorical(y) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [66]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

In [71]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the model
model = Sequential()

# Input layer with Batch Normalization and a slightly higher regularization
model.add(Dense(1024, kernel_regularizer=l2(0.005), input_shape=(1024,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(Dropout(0.5))

# Hidden layers with Batch Normalization, increased neurons, and Dropout
model.add(Dense(1024, kernel_regularizer=l2(0.005)))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(Dropout(0.5))

model.add(Dense(512, kernel_regularizer=l2(0.005)))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(Dropout(0.5))

model.add(Dense(256, kernel_regularizer=l2(0.005)))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(Dropout(0.5))

model.add(Dense(128, kernel_regularizer=l2(0.005)))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(Dropout(0.5))

model.add(Dense(64, kernel_regularizer=l2(0.005)))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Output layer
model.add(Dense(1, activation="sigmoid"))

# Compile the model
optimizer = Adam(learning_rate=0.00001)  # Further reduced learning rate
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

# Checkpoint to save the best model
checkpoint = ModelCheckpoint("final_best_model.h5", save_best_only=True, monitor='val_accuracy', mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train the model
training_results = model.fit(X_train, 
                             y_train, 
                             epochs=300, 
                             batch_size=32,
                             validation_data=(X_val, y_val),
                             callbacks=[checkpoint, early_stopping],
                             verbose=1)

Epoch 1/300
11/11 [==============================] - 4s 96ms/step - loss: 17.3713 - accuracy: 0.5149 - val_loss: 17.3203 - val_accuracy: 0.5357
Epoch 2/300
11/11 [==============================] - 0s 35ms/step - loss: 17.2457 - accuracy: 0.6369 - val_loss: 17.3162 - val_accuracy: 0.5238
Epoch 3/300
11/11 [==============================] - 0s 35ms/step - loss: 17.1795 - accuracy: 0.6935 - val_loss: 17.3115 - val_accuracy: 0.5238
Epoch 4/300
11/11 [==============================] - 0s 37ms/step - loss: 17.0882 - accuracy: 0.7917 - val_loss: 17.3008 - val_accuracy: 0.5238
Epoch 5/300
11/11 [==============================] - 0s 37ms/step - loss: 17.0421 - accuracy: 0.8274 - val_loss: 17.2913 - val_accuracy: 0.5357
Epoch 6/300
11/11 [==============================] - 0s 34ms/step - loss: 17.0085 - accuracy: 0.8542 - val_loss: 17.2747 - val_accuracy: 0.5238
Epoch 7/300
11/11 [==============================] - 1s 82ms/step - loss: 16.9587 - accuracy: 0.8869 - val_loss: 17.2544 - val_accuracy:

In [73]:
best_model = tf.keras.models.load_model("final_best_model.h5")
test_results = best_model.evaluate(X_test, y_test, verbose = 0 ) 
print(f'Test Acuuracy : {test_results[1]*100:.2f}%')
y_pred_probs = best_model.predict(X_test)
y_pred = np.round(y_pred_probs)
precision = precision_score(y_test , y_pred )
recall = recall_score (y_test , y_pred )
f1= f1_score ( y_test , y_pred )
print(f'precision: {precision:.4f}')
print(f'Recall: {recall : .4f}')
print(f'f1-score: {f1 : .4f}')

Test Acuuracy : 98.41%
4/4 [==============================] - 0s 7ms/step
precision: 0.9825
Recall:  0.9825
f1-score:  0.9825
